## Data Visualize 

In this notebook , I  am loading the data and visualizing it .

### importing packages

In [ ]:
import pickle

import matplotlib.pyplot as plt

Loading data

In [ ]:
X_train = pickle.load(open("train_images.pkl","rb"))
y_train = pickle.load(open("train_labels.pkl","rb"))

Sample image

In [ ]:
image= X_train[1717]

label= y_train[1717]

label

function for visualize image with label

In [ ]:
def visualize(image,label):
    x,y=[],[]
    for i in range(label.shape[0]):
        if (i+1)%2==1:
            x.append(label[i])
        else:
            y.append(label[i])


    plt.imshow(image.reshape(96,96),cmap="gray")
    plt.scatter(x,y,c="r")

In [ ]:
visualize(image,label)

In [ ]:
import numpy as np

In [ ]:
np.save("image.npy",image)

In [ ]:
image.shape

In [ ]:
cv2.imwrite("sample.jpg",image)

In [ ]:
import glob
import os
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.image as mpimg
import pandas as pd
import cv2
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils


    

    
# tranforms

class Normalize(object):
    """Convert a color image to grayscale and normalize the color range to [0,1]."""        

    def __call__(self, sample):
        image, key_pts = sample['image'], sample['keypoints']
        
        image_copy = np.copy(image)
        key_pts_copy = np.copy(key_pts)

        # convert image to grayscale
        image_copy=  image_copy / 255.0
            
        
        # scale keypoints to be centered around 0 with a range of [-1, 1]
        # mean = 100, sqrt = 50, so, pts should be (pts - 100)/50
#         key_pts_copy = (key_pts_copy - 100)/50.0
        key_pts_copy = (key_pts_copy - 48) / 48.0 # NaimishNet paper p3 Data Pre-processing

        return {'image': image_copy, 'keypoints': key_pts_copy}


class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, key_pts = sample['image'], sample['keypoints']

        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = cv2.resize(image, (new_w, new_h))
        
        # scale the pts, too
        key_pts = key_pts * [new_w / w, new_h / h]

        return {'image': img, 'keypoints': key_pts}


class RandomCrop(object):
    """Crop randomly the image in a sample.

    Args:
        output_size (tuple or int): Desired output size. If int, square crop
            is made.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        image, key_pts = sample['image'], sample['keypoints']

        h, w = image.shape[:2]
        new_h, new_w = self.output_size
        print(h,w)
        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)

        image = image[top: top + new_h,
                      left: left + new_w]

        key_pts = key_pts - [left, top]

        return {'image': image, 'keypoints': key_pts}


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, key_pts = sample['image'], sample['keypoints']
         
        # if image has no grayscale color channel, add one
        if(len(image.shape) == 2):
            # add that third color dim
            image = image.reshape(image.shape[0], image.shape[1], 1)
            
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        
        return {'image': torch.from_numpy(image),
                'keypoints': torch.from_numpy(key_pts)}

class FacialKeypointsDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, image_pickle_file,label_pickle_file, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.images= pickle.load(open(image_pickle_file,"rb"))
        self.labels= pickle.load(open(label_pickle_file,"rb"))
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        if True:
            image=self.images[idx]
            keypoints=self.labels[idx]
            sample = {'image': image, 'keypoints': keypoints}

            if self.transform:
                sample = self.transform(sample)

            return sample
        
            
            #print(e)
 



# Perform data augmentation on train dataset by first scaling and then random cropping
train_data_transform = transforms.Compose([Normalize(), \
                                           ToTensor()])


In [ ]:
dataset = FacialKeypointsDataset("train_images.pkl","train_labels.pkl",transform=train_data_transform)

In [ ]:
sample=dataset[0]

In [ ]:
image=sample["image"]

In [ ]:
label=sample["keypoints"]

In [ ]:
label

In [ ]:
plt.imshow(image.reshape(96,96),cmap="gray")

In [ ]:
keypoints=sample["keypoints"]

In [ ]:
keypoints